In [1]:
### IMPORTAR DEPENDENCIAS ###
import streamlit as st
from PyPDF2 import PdfReader, PdfFileReader
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from typing import List
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from pydantic import BaseModel, Field
import dotenv
import os
import regex as re

import warnings
warnings.filterwarnings('ignore')

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

c:\anaconda\anaconda3\envs\RAG_simple\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# carga variables de entorno de .env
dotenv.load_dotenv()

True

In [3]:
# extrae texto de PDF
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# extrae text chunks (pedazos de texto)
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['(VOLUME\s+\d+\s+-\s+Pag\s+\d+\s+)'],
        is_separator_regex=True,
        keep_separator=True,
        chunk_size=2048, 
        chunk_overlap=256,
        )
    chunk_OK = False
    ajuste = 0
    cont = 0
    print('Chunking: procesando texto...')
    while not chunk_OK:
        try:
            chunks = text_splitter.split_text(text[-(len(text)-ajuste):])
            print('Texto procesado correctamente')
            chunk_OK = True
        except:
            cont += 1
            print(f'Ajustando largo de texto. Iteración {cont}')
            ajuste += 50
        
    chunks = [chunk for chunk in chunks if len(chunk)>50]
    
    return chunks

# vector store chromadb
def get_vector_store(text_chunks):
    Chroma.from_texts(texts=text_chunks,
                          embedding=OpenAIEmbeddings(
                              model='text-embedding-3-large'), 
                          persist_directory="./chroma_db",
                          )
    
# vector store pinecone
def vector_store_pinecone(text_chunks):
    PineconeVectorStore.from_documents(
        documents=text_chunks, 
        embedding=OpenAIEmbeddings(), 
        index_name=os.getenv('PINECONE_INDEX_NAME'))

### Archivos individuales

In [7]:
os.getcwd()

'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o'

ruta de acceso a archivos de texto

C:\Users\CarlosGorricho(ext)\Documents\Proyecto AI TN\01_chatbot

TNAI_chatbot - GPT 4o\raw_text_VOLUME 2 - INSTRUCTION BOOK VOL II.txt

In [8]:
file = 'C:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\raw_text_VOLUME 3 - INSTRUCTION BOOK VOL III.txt'

In [9]:
raw_text = open(file, 'r').read()

In [ ]:
type(raw_text)

str

In [ ]:
len(raw_text)

834802

In [14]:
get_text_chunks(raw_text)

Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente


["UCTION BOOK VOLUME Ipp (QJLQH W\\SH + ')9  \n \xa0 \xa0 \xa0 \xa0\xa0\xa0 \xa0\xa0\xa0 \xa0 \xa0\xa0 \xa0 \xa0 \xa0\xa0\xa0\xa0\xa0 \xa0 \xa0 \xa0\xa0 \xa0 +\\XQGDL *OREDO 6HUYLFH &R /WG WK )ORRU &HQWXP 6FLHQFH 3DUN &HQWXP MXQJDQJ UR  \n \xa0 \xa0\xa0 \xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 +DHXQGDH JX %XVDQ .RUHD KWWS ZZZ K\\XQGDL JV FRP                       \n \n \xa0\xa0\xa0\xa0 \xa0 \xa0 \xa0\xa0 \xa0:DUUDQW\\ 6HUYLFH 7(/   \n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n\xa0\xa0\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 0DULQH        \n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n \xa0\xa0 \xa0\xa0 \xa0 6WDWLRQDU\\ )$;   \n \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0 \xa0\xa0 \xa0 \xa0 \xa0 ( PDLO VHUYLFH#K\\XQGDL JV FRP  \n \n \xa0 \xa0 \xa0\xa0 \xa03DUWV 6DOHV 7(/   \n\xa0\xa0\xa0\xa0\xa

In [ ]:
print(chunks[-1])

VOLUME 2 - Pag 901 
Article N°PageItem1800-06422 94971800-06440 104471800-06441 98331800-06441 
103331800-06446 94571800-06464 94931800-06488 925 - 61800-06498 94511800-06526 
98381800-06908 106261800-06908 107261800-06920 98221800-06920101221800-07065 
104451800-07067 104431800-07144 103321800-07145 103321800-07147 103321800-07149 
103321800-07150 103311800-07151 103311800-07490 106321800-07490 107321800-07617 
104531800-07677 98211800-07677101211800-07683 106281800-07683 107281800-07685 
106241800-07701 9271800-07751 104121800-08118 106221800-08118 107221800-08150 
98381800-08246 106291800-08246 107291800-08247 10630 
 
Article N°PageItem1800-08247 107301800-08255 106271800-08255 107271800-08751 
104171800-08950 104381800-09546 10411800-09654 10431800-09654 10731800-09660 
10421800-09660 10621800-09727 10441800-09734 94911800-09950 94451800-09950 
10401800-10063 94901800-10066 94901800-10072 94911800-10202 92151800-10203 
92151800-10225 94461800-19296 102-1800-19305 102-1800-19306 10

In [ ]:
len(chunks)

438

In [ ]:
for i, chunk in enumerate(chunks):
    print(f'{i}. Largo del chunk: {len(chunk)}')

0. Largo del chunk: 1171
1. Largo del chunk: 1501
2. Largo del chunk: 1851
3. Largo del chunk: 1117
4. Largo del chunk: 4554
5. Largo del chunk: 2946
6. Largo del chunk: 2297
7. Largo del chunk: 701
8. Largo del chunk: 2199
9. Largo del chunk: 2004
10. Largo del chunk: 1335
11. Largo del chunk: 1926
12. Largo del chunk: 1602
13. Largo del chunk: 1704
14. Largo del chunk: 1518
15. Largo del chunk: 1745
16. Largo del chunk: 840
17. Largo del chunk: 1938
18. Largo del chunk: 1704
19. Largo del chunk: 1998
20. Largo del chunk: 1346
21. Largo del chunk: 975
22. Largo del chunk: 1942
23. Largo del chunk: 767
24. Largo del chunk: 1700
25. Largo del chunk: 1798
26. Largo del chunk: 1216
27. Largo del chunk: 1822
28. Largo del chunk: 2350
29. Largo del chunk: 59
30. Largo del chunk: 2304
31. Largo del chunk: 1119
32. Largo del chunk: 2026
33. Largo del chunk: 1026
34. Largo del chunk: 1980
35. Largo del chunk: 3770
36. Largo del chunk: 2360
37. Largo del chunk: 471
38. Largo del chunk: 1737
39.

### Todos los archivos

In [6]:
os.getcwd()

'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o'

In [6]:
from pathlib import Path
path = os.getcwd() + '\\manuales\\02 - simples - ingles\\'
pdf_docs = [path + file for file in os.listdir(path)]
pdf_docs

['c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 1 - INSTRUCTION BOOK VOL I.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 10 - ELECTRICAL & CONTROL I.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 2 - INSTRUCTION BOOK VOL II.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 3 - INSTRUCTION BOOK VOL III.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 4 - INSTRUCTION BOOK VOL IV.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 6 - MECHANICAL EQUIPMENT II.pdf',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\TNAI_chatbot - GPT 4o\\manuales\\02 - simples - ingles\\VOLUME 7 - MECHANICAL EQUIPMENT III.pdf',
 'c:\

In [ ]:
len(pdf_docs)

9

In [ ]:
type(pdf_docs)

list

In [ ]:
pdf_docs[1].split('.')[0].split('\\')[-1]

'VOLUME 10 - ELECTRICAL & CONTROL I'

In [ ]:
for doc in pdf_docs:
    file_name = doc.split('.')[0].split('\\')[-1]
    print(f'Procesando manual: {file_name}')
    raw_text_file = get_pdf_text([doc])
    with open(f'raw_text_{file_name}.txt', 'w') as file:
        file.write(raw_text_file)


Procesando manual: VOLUME 1 - INSTRUCTION BOOK VOL I
Procesando manual: VOLUME 10 - ELECTRICAL & CONTROL I
Procesando manual: VOLUME 2 - INSTRUCTION BOOK VOL II
Procesando manual: VOLUME 3 - INSTRUCTION BOOK VOL III
Procesando manual: VOLUME 4 - INSTRUCTION BOOK VOL IV
Procesando manual: VOLUME 6 - MECHANICAL EQUIPMENT II
Procesando manual: VOLUME 7 - MECHANICAL EQUIPMENT III
Procesando manual: VOLUME 8 - MECHANICAL EQUIPMENT IV
Procesando manual: VOLUME 9 - MECHANICAL EQUIPMENT V


Carga texto de los manuales

In [7]:
raw_text_dict = {}
    
for doc in pdf_docs:
    file_name = doc.split('.')[0].split('\\')[-1]
    print(f'Cargando manual: {file_name}')
    raw_text_dict[file_name] = open(f'raw_text_{file_name}.txt', 'r').read()

Cargando manual: VOLUME 1 - INSTRUCTION BOOK VOL I
Cargando manual: VOLUME 10 - ELECTRICAL & CONTROL I
Cargando manual: VOLUME 2 - INSTRUCTION BOOK VOL II
Cargando manual: VOLUME 3 - INSTRUCTION BOOK VOL III
Cargando manual: VOLUME 4 - INSTRUCTION BOOK VOL IV
Cargando manual: VOLUME 6 - MECHANICAL EQUIPMENT II
Cargando manual: VOLUME 7 - MECHANICAL EQUIPMENT III
Cargando manual: VOLUME 8 - MECHANICAL EQUIPMENT IV
Cargando manual: VOLUME 9 - MECHANICAL EQUIPMENT V


In [8]:
type(raw_text_dict)

dict

In [9]:
print(raw_text_dict.keys())

dict_keys(['VOLUME 1 - INSTRUCTION BOOK VOL I', 'VOLUME 10 - ELECTRICAL & CONTROL I', 'VOLUME 2 - INSTRUCTION BOOK VOL II', 'VOLUME 3 - INSTRUCTION BOOK VOL III', 'VOLUME 4 - INSTRUCTION BOOK VOL IV', 'VOLUME 6 - MECHANICAL EQUIPMENT II', 'VOLUME 7 - MECHANICAL EQUIPMENT III', 'VOLUME 8 - MECHANICAL EQUIPMENT IV', 'VOLUME 9 - MECHANICAL EQUIPMENT V'])


Combina texto de todos los archivos

In [10]:
raw_text = ''
largo = 0

for key in raw_text_dict.keys():
    text = raw_text_dict[key]
    print(f'Longitud de {key}: {len(text)}')
    largo += len(text)
    raw_text += text

print(f'{largo}, {len(raw_text)}, {largo == len(raw_text)}')

Longitud de VOLUME 1 - INSTRUCTION BOOK VOL I: 428313
Longitud de VOLUME 10 - ELECTRICAL & CONTROL I: 2583997
Longitud de VOLUME 2 - INSTRUCTION BOOK VOL II: 1143960
Longitud de VOLUME 3 - INSTRUCTION BOOK VOL III: 834802
Longitud de VOLUME 4 - INSTRUCTION BOOK VOL IV: 261145
Longitud de VOLUME 6 - MECHANICAL EQUIPMENT II: 1534497
Longitud de VOLUME 7 - MECHANICAL EQUIPMENT III: 1375191
Longitud de VOLUME 8 - MECHANICAL EQUIPMENT IV: 1415858
Longitud de VOLUME 9 - MECHANICAL EQUIPMENT V: 1574585
11152348, 11152348, True


In [11]:
chunks = get_text_chunks(raw_text)
len(chunks)

Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente


6019

In [13]:
text_chunks = {}

for key in raw_text_dict.keys():
    print(f'Procesando {key}')
    text_chunks[key] = get_text_chunks(raw_text_dict[key])
    print(f'    {len(text_chunks[key])} chunks')
    

Procesando VOLUME 1 - INSTRUCTION BOOK VOL I
Chunking: procesando texto...
Texto procesado correctamente
    284 chunks
Procesando VOLUME 10 - ELECTRICAL & CONTROL I
Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente
    1504 chunks
Procesando VOLUME 2 - INSTRUCTION BOOK VOL II
Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente
    634 chunks
Procesando VOLUME 3 - INSTRUCTION BOOK VOL III
Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente
    438 chunks
Procesando VOLUME 4 - INSTRUCTION BOOK VOL IV
Chunking: procesando texto...
Texto procesado correctamente
    140 chunks
Procesando VOLUME 6 - MECHANICAL EQUIPMENT II
Chunking: procesando texto...
Texto procesado correctamente
    777 chunks
Procesando VOLUME 7 - MECHANICAL EQUIPMENT III
Chunking: procesando texto...
Ajustando largo de texto. Iteración 1
Texto procesado correctamente
    675 chunks

In [14]:
list(text_chunks.keys())

['VOLUME 1 - INSTRUCTION BOOK VOL I',
 'VOLUME 10 - ELECTRICAL & CONTROL I',
 'VOLUME 2 - INSTRUCTION BOOK VOL II',
 'VOLUME 3 - INSTRUCTION BOOK VOL III',
 'VOLUME 4 - INSTRUCTION BOOK VOL IV',
 'VOLUME 6 - MECHANICAL EQUIPMENT II',
 'VOLUME 7 - MECHANICAL EQUIPMENT III',
 'VOLUME 8 - MECHANICAL EQUIPMENT IV',
 'VOLUME 9 - MECHANICAL EQUIPMENT V']

In [15]:
len(text_chunks['VOLUME 1 - INSTRUCTION BOOK VOL I'])

284

In [16]:
text_chunks['VOLUME 1 - INSTRUCTION BOOK VOL I']

['VOLUME 1 - Pag 1 \nVOLUME 1 - Pag 1 \nHYUNDAI \n \nINSTRUCTION BOOK VOLUME I  Engine type H35DFV  Hyundai Global Service Co., Ltd. 6th \nFloor, Centum Science Park, 79, Centum jungang-ro,  Haeundae-gu, Busan, Korea, 48058\nhttp://www.hyundai-gs.com \n \naWarranty Service TEL :  (82) 52-204-7887(Marine)        (82) \n52-204-7742(Stationary) FAX :  (82) 52-204-7801 E-mail : service@hyundai-gs.com \n \nParts Sales TEL :  (82) 52-204-7718(Marine)        (82) 52-204-7742(Stationary)  FAX\n:  (82) 52-204-7700 E-mail : sales@hyundai-gs.com \n \n-------------------------------------------------------------------------------- \nVOLUME 1 - Pag 2 \nVOLUME 1 - Pag 2 \nThis book must in no case be wholly or partially copied and publishedor passed to \nunauthorized persons without the permission ofHYUNDAI HEAVY INDUSTIRES CO., LTD. \nENGINE & MACHINERYDIVISION. \n \n-------------------------------------------------------------------------------- \nVOLUME 1 - Pag 3 \nVOLUME 1 - Pag 3',
 'VOLUME 1 -

Creación del vector store

In [18]:
len(chunks)

6019

In [19]:
get_vector_store(chunks)

Consulta al vector store

In [21]:
global_llm = "gpt-4o"

llm = ChatOpenAI(
    model=global_llm, 
    temperature=0,
    )

In [24]:
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large'
    )

new_db = Chroma(persist_directory="./chroma_db",
                embedding_function=embeddings,
                )


In [26]:
retriever = new_db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'fetch_k': 50},
)

In [149]:
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=new_db.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 5, 'fetch_k': 50},
    ), 
    llm=llm
)

In [30]:
template = """Use the following pieces of context to answer the question at the end. Always answer the question in the language in which it is asked. If you don't know the answer, just say that you don't know, don't try to make up an answer. Give your responses primarily in numbered lists.
    
{context}

Question: {question}

Helpful Answer:"""

rag_prompt_custom = PromptTemplate.from_template(template)

rag_prompt_custom_chat = ChatPromptTemplate.from_template(template)


In [42]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom 
    | llm
    | StrOutputParser()
    )

In [41]:
chain_chat = (
        {
        "context": retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom_chat
    | llm
    | StrOutputParser()
    )

In [144]:
chain_multi = (
    {
        "context": multi_retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom 
    | llm
    | StrOutputParser()
    )

In [62]:
user_question = 'which are the steps to start an engine?'

In [46]:
response = chain.invoke(user_question)
print(response)

Here are the steps to start an engine based on the provided context:

1. **Preheating the Engine**:
   - Operate the cooling water heating system.
   - Circulate heating water through the cylinders for at least 2 hours before starting.
   - Ensure the temperature of the water is over 60℃ for HFO and over 40℃ for MDO.

2. **Preparation of Fuel System**:
   - Drain fuel tanks for water and check filters.
   - Check fuel tank level and fill if needed.
   - Check the cooling system for diesel fuel oil.
   - For starting on HFO, recheck preheating of the engine ready for HFO circulation.

3. **Starting Sequence**:
   - Ensure all start requests are present at the end of the Start Delay timer.
   - Energize the fuel relay and crank the engine.
   - If the engine fails to fire, disengage the starter motor for the Crank Rest duration and attempt to start again.
   - If the engine fires, disengage the starter motor.
   - Activate the Safety On Delay timer to allow fault inputs to stabilize.

4.

In [47]:
response_chat = chain_chat.invoke(user_question)
print(response_chat)

Here are the steps to start an engine based on the provided context:

1. **Preheating the Engine**:
   - Operate the cooling water heating system.
   - Circulate heating water through the cylinders for at least 2 hours before starting.
   - Ensure the temperature of the water is over 60°C for HFO and over 40°C for MDO.

2. **Preparation of Fuel System**:
   - Drain fuel tanks for water and check filters.
   - Check fuel tank level and fill if needed.
   - Check the cooling system for diesel fuel oil.
   - For starting on HFO, recheck preheating of the engine ready for HFO circulation.

3. **Starting Sequence**:
   - Ensure all start requests are present at the end of the Start Delay timer.
   - Energize the fuel relay and crank the engine.
   - If the engine fails to fire, disengage the starter motor for the Crank Rest duration and attempt to start again.
   - If the engine fires, disengage the starter motor.
   - Activate the Safety On Delay timer to allow fault inputs to stabilize.



In [50]:
docs = retriever.invoke(user_question)
docs

[Document(page_content='VOLUME 10 - Pag 1673 \n5.4.2 STARTING SEQUENCE \n \nNOTE: If the unit has been configured for CAN, compatible ECU’s receive the start \ncommand via CAN and transmit the engine speed to the DSE controller. \n \nNOTE: For further details of module configuration, refer to DSE Publication: 057-238\n DSE8610 MKII Configuration Suite PC Software Manual.  To allow for ‘false’ start \nrequests, the Start Delay timer begins.  Should all start requests be removed during\nthe Start Delay timer, the unit returns to a stand-by state.  If a start request is \nstill present at the end of the Start Delay timer, the fuel relay is energised and \nthe engine is cranked.   If the engine fails to fire during this cranking attempt \nthen the starter motor is disengaged for the Crank Rest duration after which the \nnext start attempt is made. Should this sequence continue beyond the Set Number Of \nAttempts, the start sequence is terminated and the display shows Fail to Start.  The\ns

In [63]:
response_multi = chain_multi.invoke(user_question)
print(response_multi)

Here are the steps to start an engine based on the provided context:

1. **Preparation of Diesel Control System:**
   - Check smooth movement of fuel regulating linkages from governor to every rack of the fuel injection pump.
   - Check ‘Stop’ and ‘Max’ position for both Governor Load Index and Fuel Pump rack.
   - Check function of ‘Emergency Stop’ button and reset.
   - Push the handle on fuel regulating shaft to ‘Max’ position and press ‘Emergency Stop’ button, then every rack of fuel injection pump should be extruded to ‘Stop’ position.
   - Check flexibility of each rack of fuel injection pump.
   - Check mechanical fuel limiter for overload (Set to 110% load) and for start (Set to about 30 ~ 50% load).

2. **Air Running:**
   - Check air intake and exhaust gas passages are free.
   - Check ‘Start Ready’ lamp ‘ON’.
   - Choose “Starting Valve” at test mode of LOP (Local Operating Panel) and press “Switch On” button.
   - Check air running condition to ensure pre-lubrication and re

In [64]:
docs_multi = multi_retriever.invoke(user_question)
docs_multi

[Document(page_content='VOLUME 10 - Pag 1673 \n5.4.2 STARTING SEQUENCE \n \nNOTE: If the unit has been configured for CAN, compatible ECU’s receive the start \ncommand via CAN and transmit the engine speed to the DSE controller. \n \nNOTE: For further details of module configuration, refer to DSE Publication: 057-238\n DSE8610 MKII Configuration Suite PC Software Manual.  To allow for ‘false’ start \nrequests, the Start Delay timer begins.  Should all start requests be removed during\nthe Start Delay timer, the unit returns to a stand-by state.  If a start request is \nstill present at the end of the Start Delay timer, the fuel relay is energised and \nthe engine is cranked.   If the engine fails to fire during this cranking attempt \nthen the starter motor is disengaged for the Crank Rest duration after which the \nnext start attempt is made. Should this sequence continue beyond the Set Number Of \nAttempts, the start sequence is terminated and the display shows Fail to Start.  The\ns

In [66]:
for doc in docs_multi:
    print(len(doc.page_content))

1773
2148
1658
1467
2385
1659
3214
2162
2053
2972
148
188
200
1454
251
1139
2046
1603
167
1254
1770
821
918
1897
1524
1381
2087
2216
1283


In [67]:
user_question = 'cuales son los pasos para arrancar un motor?'


In [68]:
response = chain.invoke(user_question)
print(response)

Aquí tienes los pasos para arrancar un motor, basados en los documentos proporcionados:

1. **Preparativos Iniciales:**
   - Asegúrate de que el disyuntor esté apagado para evitar descargas eléctricas.
   - Verifica que no haya nadie cerca del generador y advierte al personal cercano.
   - No operes el generador con las cubiertas protectoras removidas.
   - Confirma el voltaje de la batería y no uses el arrancador por más de 10 segundos a la vez, permitiendo 30 segundos de enfriamiento entre intentos.

2. **Secuencia de Arranque:**
   - Si la unidad está configurada para CAN, los ECU compatibles recibirán el comando de arranque a través de CAN y transmitirán la velocidad del motor al controlador DSE.
   - El temporizador de retardo de arranque comienza para permitir solicitudes de arranque "falsas".
   - Si la solicitud de arranque persiste al final del temporizador de retardo, el relé de combustible se energiza y el motor se arranca.
   - Si el motor no arranca en el primer intento, e

In [69]:
response_chat = chain_chat.invoke(user_question)
print(response_chat)

Aquí tienes los pasos para arrancar un motor, basados en el contexto proporcionado:

1. **Preparativos Iniciales:**
   - Asegúrate de que el interruptor del circuito esté apagado para evitar descargas eléctricas.
   - Verifica que no haya nadie cerca del generador y advierte al personal cercano sobre el arranque.
   - No operes el generador con las cubiertas protectoras, cubiertas de acceso o cubiertas de la caja de terminales removidas.

2. **Verificación de la Batería:**
   - Confirma el voltaje de la batería. No utilices el arrancador por más de 10 segundos a la vez y permite 30 segundos de enfriamiento antes de usarlo nuevamente.

3. **Secuencia de Arranque:**
   - Si la unidad está configurada para CAN, las ECU compatibles recibirán el comando de arranque a través de CAN y transmitirán la velocidad del motor al controlador DSE.
   - El temporizador de retardo de arranque comienza para permitir solicitudes de arranque "falsas". Si todas las solicitudes de arranque se eliminan duran

In [70]:
docs = retriever.invoke(user_question)
docs

[Document(page_content='VOLUME 10 - Pag 1527 [STARTING]    - Make sure the circuit breaker is OFF electrical shock can cause \nsevere personal injury or       death. - Make sure there is no one next to generator\nset. Warn any personnel nearby that you are     starting up. - Do not operate the \ngenerator set with protective covers, access covers or terminal box covers  removed.\n When the engine does not starts, confirm battery voltage  ( We does not include The\nAutomatic Battery Charger basically - OPTION)  Do not run the starter for more than \n10 seconds at a time. Then allow 30 seconds for cooling before  using it again.     \n[WARM-UP RUN]    - Load periods of engine warm-up is not recommended. This can \nresult in carbon deposits in the     combustion chambers and incomplete fuel \nburning. - If the engine operates abnormally, stop it as soon as possible, try to \nfind a problem and its source, then make necessary repairs before starting it again.\n \n▣ Warm up the engine for 5

In [71]:
response_multi = chain_multi.invoke(user_question)
print(response_multi)

Aquí tienes los pasos para arrancar un motor, basados en la información proporcionada:

1. **Preparaciones Previas:**
   - Verifica el nivel de combustible, nivel de refrigerante y nivel de aceite del motor. Reabastece si es necesario.
   - Asegúrate de que el nivel de aceite no supere la marca máxima en la varilla de medición.
   - Utiliza combustibles, lubricantes y refrigerantes aprobados.

2. **Procedimiento de Arranque:**
   - Inserta la llave en el interruptor de arranque.
   - Mueve la palanca de control a "Velocidad de ralentí".
   - Gira el interruptor de la llave en el sentido de las agujas del reloj.
   - No operes el motor de arranque por más de 10 segundos a la vez.
   - Después de la ignición del motor, suelta la llave.
   - Ajusta la palanca de control a la velocidad deseada.
   - Si el motor no arranca, suelta la llave, espera aproximadamente 1 minuto y vuelve a intentar.

3. **Consideraciones Adicionales:**
   - Evita hacer funcionar el motor en frío durante mucho tiem

In [72]:
docs_multi = multi_retriever.invoke(user_question)
docs_multi

[Document(page_content='VOLUME 10 - Pag 1776 \nVOLUME 10 - Pag 1776 \n4. COMMISSIONING AND OPERATION4.1. PreparationsBefore daily starting the engine, \ncheck fuel level, coolant level and engine oil level andreplenish, if necessary.The \nnotches in the dipstick indicate the highest and lowest permissible oil \nlevels.CAUTION : Do not add so much engine oil that the oil level rises above the \nmax. marking onthe dipstick. Overfilling will result in damage to the engine.The oil\nrequired in the sump is specified in the “Engine Specification” at the head of \ntheseinstructions.NOTE : The oil required to fill the oil filters and pipes depends \nupon the engine equipment anduse and must be determined individually at the time of \ninitial commissioning. (Make anote of the determined quantity)Ensure outpost \ncleanliness when handling fuels, lubricants and coolants.Use approved fuels, \nlubricants and coolants only, as otherwise the manufacturer’s guaranteewill be null \nand void.4.2. Starti

### Crea consulta con fuente de documentos

In [164]:
user_question = 'cuales son los pasos para poner en marcha un motor?'


In [165]:
from langchain_core.runnables import RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | rag_prompt_custom
    | llm
    | StrOutputParser()
    )

rag_chain_with_source = RunnableParallel(
    {"context": multi_retriever, "question": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)

response_source = rag_chain_with_source.invoke(user_question)

In [166]:
response_source

{'context': [Document(page_content='VOLUME 10 - Pag 1673 \n5.4.2 STARTING SEQUENCE \n \nNOTE: If the unit has been configured for CAN, compatible ECU’s receive the start \ncommand via CAN and transmit the engine speed to the DSE controller. \n \nNOTE: For further details of module configuration, refer to DSE Publication: 057-238\n DSE8610 MKII Configuration Suite PC Software Manual.  To allow for ‘false’ start \nrequests, the Start Delay timer begins.  Should all start requests be removed during\nthe Start Delay timer, the unit returns to a stand-by state.  If a start request is \nstill present at the end of the Start Delay timer, the fuel relay is energised and \nthe engine is cranked.   If the engine fails to fire during this cranking attempt \nthen the starter motor is disengaged for the Crank Rest duration after which the \nnext start attempt is made. Should this sequence continue beyond the Set Number Of \nAttempts, the start sequence is terminated and the display shows Fail to St

In [119]:
response_source['question']

'cuales son los pasos para poner en marcha un motor'

In [120]:
response_source['answer']

'1. **Preparaciones Previas:**\n   - Verificar el nivel de combustible, nivel de refrigerante y nivel de aceite del motor.\n   - Asegurarse de que el nivel de aceite no supere la marca máxima en la varilla de medición.\n   - Utilizar combustibles, lubricantes y refrigerantes aprobados.\n\n2. **Inicio del Motor:**\n   - Insertar la llave en el interruptor de arranque.\n   - Mover la palanca de control a "Velocidad de ralentí".\n   - Girar el interruptor de llave en el sentido de las agujas del reloj.\n   - No operar el motor de arranque por más de 10 segundos a la vez.\n   - Después de la ignición del motor, soltar la llave.\n   - Ajustar la palanca de control a la velocidad deseada.\n   - Si el motor no arranca, soltar la llave, esperar aproximadamente 1 minuto y luego intentar nuevamente.\n\n3. **Consideraciones Adicionales:**\n   - Evitar hacer funcionar el motor en frío durante mucho tiempo para prevenir el desgaste por corrosión.\n   - En el primer arranque de un motor revisado o d

In [121]:
response_source['context']

[Document(page_content='VOLUME 10 - Pag 1527 [STARTING]    - Make sure the circuit breaker is OFF electrical shock can cause \nsevere personal injury or       death. - Make sure there is no one next to generator\nset. Warn any personnel nearby that you are     starting up. - Do not operate the \ngenerator set with protective covers, access covers or terminal box covers  removed.\n When the engine does not starts, confirm battery voltage  ( We does not include The\nAutomatic Battery Charger basically - OPTION)  Do not run the starter for more than \n10 seconds at a time. Then allow 30 seconds for cooling before  using it again.     \n[WARM-UP RUN]    - Load periods of engine warm-up is not recommended. This can \nresult in carbon deposits in the     combustion chambers and incomplete fuel \nburning. - If the engine operates abnormally, stop it as soon as possible, try to \nfind a problem and its source, then make necessary repairs before starting it again.\n \n▣ Warm up the engine for 5

In [169]:
all_pages = []

for doc in response_source['context']:
    print('**'*50)
    page = ((re.findall('(VOLUME\s+\d+\s+-\s+Pag\s+\d+)', doc.page_content))) 
    print(page)
    all_pages.extend(page)

source_pages = '\n'.join([pg for pg in sorted(set(all_pages))])

print(source_pages)


****************************************************************************************************
['VOLUME 10 - Pag 1673', 'VOLUME 10 - Pag 1674', 'VOLUME 10 - Pag 1674']
****************************************************************************************************
['VOLUME 8 - Pag 1093']
****************************************************************************************************
['VOLUME 2 - Pag 32']
****************************************************************************************************
['VOLUME 6 - Pag 112', 'VOLUME 6 - Pag 113', 'VOLUME 6 - Pag 113']
****************************************************************************************************
['VOLUME 2 - Pag 584']
****************************************************************************************************
['VOLUME 1 - Pag 43', 'VOLUME 1 - Pag 44', 'VOLUME 1 - Pag 44']
****************************************************************************************************
['VOLUME 10 - Pag 43

In [168]:
response_string = f"{response_source['answer']}\n\nFuente (Source)\n\n{source_pages}"
print(response_string)

Los pasos para poner en marcha un motor pueden variar dependiendo del tipo de motor y del sistema específico. A continuación, se presentan algunos pasos generales basados en el contexto proporcionado:

1. **Precalentamiento del Motor**:
   - Operar el sistema de calentamiento del agua de enfriamiento.
   - Asegurarse de que el agua de calentamiento circule a través de los cilindros durante al menos 2 horas antes de arrancar.
   - La temperatura del agua debe ser adecuada según el tipo de combustible (HFO: >60°C, MDO: >40°C).

2. **Preparación del Sistema de Combustible**:
   - Drenar los tanques de combustible para eliminar agua y verificar los filtros.
   - Comprobar el nivel del tanque de combustible y rellenar si es necesario.
   - Verificar el sistema de enfriamiento para el aceite diesel.
   - Para arranques en HFO, asegurarse de que el motor esté precalentado y listo para la circulación de HFO.

3. **Secuencia de Arranque**:
   - Si el motor está configurado para CAN, las ECU com

In [155]:
response_string

'1. **Preparaciones Previas:**\n   - Verificar el nivel de combustible, nivel de refrigerante y nivel de aceite del motor.\n   - Asegurarse de que el nivel de aceite no supere la marca máxima en la varilla de medición.\n   - Utilizar combustibles, lubricantes y refrigerantes aprobados.\n\n2. **Inicio del Motor:**\n   - Insertar la llave en el interruptor de arranque.\n   - Mover la palanca de control a "Velocidad de ralentí".\n   - Girar el interruptor de llave en el sentido de las agujas del reloj.\n   - No operar el motor de arranque por más de 10 segundos a la vez.\n   - Después de la ignición del motor, soltar la llave.\n   - Ajustar la palanca de control a la velocidad deseada.\n   - Si el motor no arranca, soltar la llave, esperar aproximadamente 1 minuto y luego intentar nuevamente.\n\n3. **Verificación Durante el Arranque:**\n   - Comprobar la presión del aceite del motor.\n   - Verificar la temperatura del agua.\n   - Revisar el medidor de voltaje de la batería.\n   - Observar

In [156]:
from IPython.display import display, Markdown, Latex
display(Markdown(response_string))

1. **Preparaciones Previas:**
   - Verificar el nivel de combustible, nivel de refrigerante y nivel de aceite del motor.
   - Asegurarse de que el nivel de aceite no supere la marca máxima en la varilla de medición.
   - Utilizar combustibles, lubricantes y refrigerantes aprobados.

2. **Inicio del Motor:**
   - Insertar la llave en el interruptor de arranque.
   - Mover la palanca de control a "Velocidad de ralentí".
   - Girar el interruptor de llave en el sentido de las agujas del reloj.
   - No operar el motor de arranque por más de 10 segundos a la vez.
   - Después de la ignición del motor, soltar la llave.
   - Ajustar la palanca de control a la velocidad deseada.
   - Si el motor no arranca, soltar la llave, esperar aproximadamente 1 minuto y luego intentar nuevamente.

3. **Verificación Durante el Arranque:**
   - Comprobar la presión del aceite del motor.
   - Verificar la temperatura del agua.
   - Revisar el medidor de voltaje de la batería.
   - Observar el medidor de presión.

4. **Calentamiento del Motor:**
   - Calentar el motor durante 5 a 10 minutos.
   - Durante el calentamiento, verificar la presión del aceite, la temperatura del agua y el voltaje de la batería.
   - Escuchar sonidos y vibraciones anormales. Si se detectan, detener el motor presionando el botón de parada de emergencia.

5. **Operación Normal:**
   - Asegurarse de que el motor funcione sin problemas y sin ruidos o vibraciones anormales.
   - Monitorear continuamente los indicadores de presión de aceite, temperatura del agua y voltaje de la batería.

6. **Parada del Motor:**
   - Para detener el motor, presionar el botón de parada de emergencia si es necesario.

Estos pasos aseguran un arranque seguro y eficiente del motor, minimizando el riesgo de daños y garantizando un funcionamiento óptimo.

Fuente (Source)

VOLUME 10 - Pag 1515 

VOLUME 10 - Pag 1527 
VOLUME 10 - Pag 1528 

VOLUME 10 - Pag 1776 

VOLUME 10 - Pag 437 
VOLUME 10 - Pag 558 

VOLUME 2 - Pag 584 

VOLUME 6 - Pag 332 

VOLUME 7 - Pag 54 

VOLUME 7 - Pag 55 

VOLUME 8 - Pag 1093 


In [ ]:
response_source['question']

In [ ]:
print(response_source['answer'])